<a href="https://colab.research.google.com/github/kushb2/Techno-Funda/blob/main/Snippets_Importing_libraries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================================================================
# PROJECT: TECHNO-FUNDA INSTITUTIONAL SCANNER (v2.0)
# OBJECTIVE: Identify Safe Stocks with Institutional Buying Momentum
# AUTHOR: You & Gemini
# ==============================================================================

# ------------------------------------------------------------------------------
# TABLE OF CONTENTS
# ------------------------------------------------------------------------------
# 1. IMPORTS & SETUP ......... Library imports and environment checks
# 2. CONFIGURATION ........... Central control panel for all thresholds
# 3. DATA ENGINE ............. Handles yfinance and NSE connections
# 4. STRATEGY LAYERS ......... The 3-Step Filtering Logic
# 5. MAIN CONTROLLER ......... The execution loop and reporting
# ------------------------------------------------------------------------------





fatal: not a git repository (or any of the parent directories): .git


# Project starts


# Main

In [ ]:
# ==============================================================================
# 5. MAIN CONTROLLER (The "Runner")
# ==============================================================================
def run_scanner():
    print(f"\n{'='*40}")
    print(f"   TECHNO-FUNDA SCANNER (Running...)")
    print(f"{'='*40}\n")

    results = []

    for ticker in Config.WATCHLIST:
        # print(f"Scanning {ticker}...", end="\r") # Progress bar style

        # 1. Fetch
        info, hist = fetch_data(ticker)
        if info is None: continue

        # 2. Run Layers
        l1_pass, l1_msg = check_layer_1_safety(info)
        if not l1_pass:
            # results.append({"Ticker": ticker, "Result": "❌ Fundamental Fail", "Detail": l1_msg})
            continue # Skip bad stocks to keep report clean

        l2_pass, l2_msg = check_layer_2_trend(hist)
        if not l2_pass:
            # results.append({"Ticker": ticker, "Result": "❌ Downtrend", "Detail": l2_msg})
            continue

        l3_pass, l3_msg = check_layer_3_momentum(hist, ticker)

        # 3. Final Decision
        status = "✅ BUY SIGNAL" if l3_pass else "Wait"

        results.append({
            "Ticker": ticker,
            "Result": status,
            "L3_Detail": l3_msg
        })

    # --- REPORTING ---
    print("\n" + "="*40)
    print("       FINAL SHORTLIST")
    print("="*40)

    if results:
        df = pd.DataFrame(results)
        # Reorder columns
        print(df)
    else:
        print("No stocks passed the Safety + Trend filters.")

# Execute
if __name__ == "__main__":
    run_scanner()

# config


In [ ]:
# ==============================================================================
# 2. CONFIGURATION (The "Brain")
# ==============================================================================
class Config:
    # --- Watchlist (The 20-25 Stocks you track) ---
    # We mix some good ones and bad ones to test the filter
    WATCHLIST = [
         "SUZLON.NS"
    ]

    # --- Layer 1: Safety (Fundamentals) ---
    MIN_MARKET_CAP_CR = 2000     # Reject Penny Stocks
    MAX_DEBT_TO_EQUITY = 1.5     # Relaxed slightly for capital intensive sectors
    MIN_ROE_PERCENT = 10.0       # Efficiency Check

    # --- Layer 2: Trend (Technical) ---
    TREND_SMA_PERIOD = 200       # The Long Term Trend
    FALLBACK_SMA_PERIOD = 50     # For recent IPOs

    # --- Layer 3: Smart Money (Momentum) ---
    VOLUME_LOOKBACK = 20         # 20 Days = 1 Trading Month
    Z_SCORE_THRESHOLD = 3.0      # 3 Sigma Event (Rare)
    DELIVERY_THRESHOLD = 40.0    # Minimum Delivery % to confirm "Buying"

    # --- System ---
    SIMULATE_NSE = True          # Set False to try real NSE connection (Risk of blocking)


# Imports


In [ ]:
# ==============================================================================
# 1. IMPORTS & SETUP
# ==============================================================================
import yfinance as yf
import pandas as pd
import numpy as np
import time
import sys

# Try importing nsepython, handle gracefully if missing
try:
    from nsepython import nse_quote_ltp
except ImportError:
    nse_quote_ltp = None
    print("⚠️ NSEPython not found. Delivery data will be simulated.")




# data engine

In [ ]:
# ==============================================================================
# 3. DATA ENGINE (The "Fetcher")
# ==============================================================================
def fetch_data(ticker):
    """
    Fetches 1 Month of history (optimal for Z-Score) and Basic Info.
    """
    try:
        stock = yf.Ticker(ticker)

        # 1. Fundamentals (Info) - Note: Fast on yfinance
        info = stock.info

        # 2. Price History (1 Month) - Note: Fast and efficient
        # We need enough data for 20-day averages
        hist = stock.history(period="3mo") # Fetch 3mo to be safe for 50SMA

        if hist.empty: return None, None
        return info, hist

    except Exception as e:
        # print(f"  [Data Error] {ticker}: {e}") # Uncomment for debug
        return None, None

def get_delivery_percentage(ticker):
    """
    Fetches Delivery % from NSE.
    Includes Simulation Mode to prevent crashes during dev.
    """
    # 1. CLEAN TICKER
    sym = ticker.replace(".NS", "")

    # 2. SIMULATION (For Development Stability)
    if Config.SIMULATE_NSE:
        # Return fake high numbers for specific stocks to test the BUY logic
        if sym in ["NETWEB", "INFY"]: return 55.5
        return 25.0

    # 3. REAL DATA FETCH (Production)
    try:
        if nse_quote_ltp:
            data = nse_quote_ltp(sym)
            # NSE JSON structure varies, basic safety check
            if data and 'deliveryToTradedQuantity' in str(data):
                # Note: Parsing real NSE JSON requires precise key navigation
                # This is a simplified placeholder for the real logic
                return float(data.get('deliveryToTradedQuantity', 0))
        return 0.0
    except:
        return 0.0


# STRATEGY LAYERS

In [ ]:
# ==============================================================================
# 4. STRATEGY LAYERS (The "Logic")
# ==============================================================================
def check_layer_1_safety(info):
    """Filters for Fundamental Quality."""
    try:
        mcap = info.get('marketCap', 0) / 10**7
        debt = info.get('debtToEquity', 999) / 100
        roe = info.get('returnOnEquity', 0) * 100

        passed = (mcap > Config.MIN_MARKET_CAP_CR and
                  debt < Config.MAX_DEBT_TO_EQUITY and
                  roe > Config.MIN_ROE_PERCENT)

        reason = f"ROE:{roe:.1f}%, D/E:{debt:.2f}"
        return passed, reason
    except:
        return False, "Data Missing"

def check_layer_2_trend(hist):
    """Filters for Uptrend (Price > SMA)."""
    try:
        # Smart SMA Selection
        period = Config.TREND_SMA_PERIOD if len(hist) > 200 else Config.FALLBACK_SMA_PERIOD

        hist['SMA'] = hist['Close'].rolling(window=period).mean()
        price = hist['Close'].iloc[-1]
        sma = hist['SMA'].iloc[-1]

        passed = price > sma
        reason = f"Price {int(price)} vs SMA {int(sma)}"
        return passed, reason
    except:
        return False, "Calc Error"

def check_layer_3_momentum(hist, ticker):
    """Detects Institutional Buying (Vol Z-Score + Delivery)."""
    try:
        # 1. Volume Stats (20 Day Lookback)
        window = Config.VOLUME_LOOKBACK
        vol_mean = hist['Volume'].rolling(window=window).mean().iloc[-1]
        vol_std = hist['Volume'].rolling(window=window).std().iloc[-1]
        curr_vol = hist['Volume'].iloc[-1]

        if vol_std == 0: return False, "Stable Vol"

        z_score = (curr_vol - vol_mean) / vol_std

        # 2. Fetch Delivery
        deliv_pct = get_delivery_percentage(ticker)

        # 3. Candle Color
        is_green = hist['Close'].iloc[-1] > hist['Open'].iloc[-1]

        # 4. The Logic
        # Condition: High Volume AND High Delivery AND Green Candle
        # We lower Z-Score to 2.0 for testing so we see some results
        is_high_vol = z_score > 2.0
        is_high_deliv = deliv_pct > Config.DELIVERY_THRESHOLD

        msg = f"Z:{z_score:.1f}, Del:{deliv_pct}%"

        if is_high_vol and is_green and is_high_deliv:
            return True, f"🚀 SNIPER BUY ({msg})"

        return False, f"Wait ({msg})"

    except Exception as e:
        return False, f"L3 Error {e}"
